In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import seaborn as sns
from scipy import stats

In [2]:
df_PAT = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Analysis & Forecasting\1. Data\MONPAT_dates.csv")
df_SAL = pd.read_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Analysis & Forecasting\1. Data\MONSAL_dates.csv")
df_PAT['PERPAI'] = pd.to_datetime(df_PAT['PERPAI'], format='%Y-%m-%d')
df_SAL['PERPAI'] = pd.to_datetime(df_SAL['PERPAI'], format='%Y-%m-%d')
df_SAL = df_SAL.sort_values(by='PERPAI', ascending=False)
df_PAT = df_PAT.sort_values(by='PERPAI', ascending=False)

In [52]:
l_cols = df_SAL.columns

In [53]:
L_prime = []
for col in df_SAL.columns :
    if (col[:5].lower() == 'prime' or col[:2].lower() == 'p.' or  col[:2].lower() == 'pr' or col[:8].lower() == 'assiduit' or col[:6].lower() == 'panier')  and (col[:4] != 'Prêt') and (col[:4] != 'PRET') :
        L_prime.append(col)

l_cols_1 = [col for col in l_cols if col not in L_prime]

In [54]:
L_Indemnité = []
for col in l_cols_1 :
    if col[:3].lower() == 'ind' : 
        L_Indemnité.append(col)
        
l_cols_2 = [col for col in l_cols_1 if col not in L_Indemnité]

In [55]:
L_Heures_supp = []
for col in l_cols_2 :
    if col[:3].lower() == 'hrs' or  col[:6].lower() == 'heures': 
        L_Heures_supp.append(col)
        
l_cols_3 = [col for col in l_cols_2 if col not in L_Heures_supp]

In [56]:
L_Retenue = []
for col in l_cols_3 :
    if col[:3].lower() == 'ret' or  col[:7].lower() == 'retenue': 
        L_Retenue.append(col)
        
l_cols_4 = [col for col in l_cols_3 if col not in L_Retenue]

In [57]:
L_Cotisation = []
for col in l_cols_4 :
    if col[:10].lower() == 'cotisation' or col[:8].lower() == 'mutuelle' or  col[:9].lower() == 'marocaine': 
        L_Cotisation.append(col)
        
l_cols_5 = [col for col in l_cols_4 if col not in L_Cotisation]

In [58]:
L_Conge = []
for col in l_cols_5 :
    if col[:5].lower() == 'congé': 
        L_Conge.append(col)
        
l_cols_6 = [col for col in l_cols_5 if col not in L_Conge]

In [59]:
L_pret_avance = []
for col in l_cols_6 :
    if col[:4].lower() == 'prêt' or col[:4].lower() == 'pret' or col[:6].lower() == 'avance' : 
        L_pret_avance.append(col)
        
l_cols_7 = [col for col in l_cols_6 if col not in L_pret_avance]

## Preapring (Contributions) Cotisations dataset 

In [50]:
# Create a function to rename duplicates
def rename_duplicate_columns(df_PAT):
    cols = pd.Series(df_PAT.columns)
    for dup in cols[cols.duplicated()].unique():
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df_PAT.columns = cols
    return df_PAT

# Rename duplicate columns
df_PAT = rename_duplicate_columns(df_PAT)

In [62]:
df_cot = df_PAT[['PERPAI']+ L_Cotisation]
df_HSupp = df_PAT[['PERPAI']+ L_Heures_supp]
df_Bonuses = df_PAT[['PERPAI']+ L_prime]
df_Indmn = df_PAT[['PERPAI']+ L_Indemnité]
df_Conge = df_PAT[['PERPAI']+ L_Conge[1:]]

In [34]:
df_pay_sums = pd.DataFrame()
# df_pay_sums['PERPAI'] = df_PAT['PERPAI']

In [64]:
df_pay_sums['Contributions_sum'] = df_cot.iloc[:,1:].sum(axis=1)
df_pay_sums['Overtime_sum'] = df_HSupp.iloc[:,1:].sum(axis=1)
df_pay_sums['Bonuses_sum'] = df_Bonuses.iloc[:,1:].sum(axis=1)
df_pay_sums['Indemnities_sum'] = df_Indmn.iloc[:,1:].sum(axis=1)
df_pay_sums['Conge_sum'] = df_Conge.iloc[:,1:].sum(axis=1)

In [42]:
df_pay_sums['PERPAI']  = df_cot['PERPAI'] 
df_pay_sums = df_pay_sums[['PERPAI','Contributions_sum','Overtime_sum','Bonuses_sum','Indemnities_sum']]

In [65]:
df_pay_sums

,PERPAI,Contributions_sum,Overtime_sum,Bonuses_sum,Indemnities_sum,Conge_sum
23,2023-12-01,-2709706.890,471559.200,1.739347e+07,5341205.331,2200123.860
22,2023-11-01,-2670869.430,1469655.590,1.617879e+07,5085585.490,2389442.020
21,2023-10-01,-2640056.590,250350.380,2.316852e+07,5086944.806,3093655.170
20,2023-09-01,-2663358.490,1654776.100,1.509127e+07,4670839.818,7057989.620
19,2023-08-01,-2660642.810,959622.970,1.545115e+07,4977829.822,7134606.220
18,2023-07-01,-2652999.520,-2521.500,2.054682e+07,5323101.468,3677728.050
17,2023-06-01,-2600657.940,-542718.738,1.584239e+07,5385057.949,2380956.530
16,2023-05-01,-2637312.900,-1295653.630,1.523117e+07,5322611.270,3210654.410
15,2023-04-01,-3234126.990,-1670864.740,4.824184e+07,5307107.479,2617048.640
14,2023-03-01,-2507275.790,-2240649.550,1.676651e+07,6858735.390,2691322.960


In [66]:
df_pay_sums.to_csv(r"C:\Users\LENOVO\Desktop\Payroll_PFE\Payroll Analysis & Forecasting\1. Data\df_analysis_contr_overtime_bonus.csv", index=False)